In [5]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.odr import ODR, Model, Data, RealData
from scipy.optimize import curve_fit
import seaborn as sns
import uncertainties as uc
from uncertainties.umath import sqrt

In [ ]:
plt.style.use('Solarize_Light2')   #Options
                                    #Solarize_Light2
                                    #_classic_test_patch
                                    #_mpl-gallery
                                    #_mpl-gallery-nogrid
                                    #bmh
                                    #classic
                                    #dark_background
                                    #fast
                                    #fivethirtyeight
                                    #ggplot
                                    #grayscale
                                    #seaborn-v0_8
                                    #seaborn-v0_8-bright
                                    #seaborn-v0_8-colorblind
                                    #seaborn-v0_8-dark
                                    #seaborn-v0_8-dark-palette
                                    #seaborn-v0_8-darkgrid
                                    #seaborn-v0_8-deep
                                    #seaborn-v0_8-muted
                                    #seaborn-v0_8-notebook
                                    #seaborn-v0_8-paper
                                    #seaborn-v0_8-pastel
                                    #seaborn-v0_8-poster
                                    #seaborn-v0_8-talk
                                    #seaborn-v0_8-ticks
                                    #seaborn-v0_8-white
                                    #seaborn-v0_8-whitegrid
                                    #tableau-colorblind10

plt.rcParams.update({

    'figure.dpi': 150,                 # Resolution of figures
    'font.size': 10,                   # Default font size
    'font.family': 'sans-serif',       # Font family
    'font.sans-serif': ['DejaVu Sans'],
    
    'axes.titlesize': 10,              # Title font size
    'axes.labelsize': 8,              # Axis label font size
    'axes.linewidth': 0.8,             # Width of the axis lines
    'axes.grid': True,                 # Show grid
    # 'grid.color': '#dddddd',
    'grid.linestyle': '--',            # Grid line style (dashed)
    'grid.linewidth': 0.5,             # Grid line width
    
    'xtick.labelsize': 8,             # X-axis tick label size
    'ytick.labelsize': 8,             # Y-axis tick label size
    'xtick.direction': 'in',           # X-axis tick direction
    'ytick.direction': 'in',           # Y-axis tick direction

    'lines.linewidth': 0.8,            # Line width for plots
     'lines.markersize': 0.3,             # Marker size for scatter plots
     'lines.marker': 'o',               # Default marker shape

    'legend.frameon': True,            # Frame around legend
    'legend.framealpha': 0.8,          # Transparency of legend frame
    'legend.fontsize': 9,             # Font size in legend
    'legend.loc': 'best',              # Default legend location
})

In [7]:
#Free Fall Data
freeFall = pd.read_csv("data/FreeFall.csv")
freeFall['t'] = freeFall['t']*.1
freeFall_Tweights = 1 / np.ones(len(freeFall['t']))**2

g=-9.81

#Incline Plane Data
InclinePlane = pd.read_csv("data/InclinePlane2.csv")
theta = np.deg2rad(22.11)
x0 = InclinePlane['x'][0]
y0 = InclinePlane['y'][0]
incline_Tweights = 1 / np.ones(len(InclinePlane['t']))**2

# If files cannot be found please replace file paths with absolute file paths.

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(12, 4))
ax[0].plot(InclinePlane['t'], InclinePlane['x'], label='Measured', color='green')
ax[0].set_title('X vs. Time')
ax[0].set_xlabel('Time (s)')
ax[0].set_ylabel('Position (m)')
ax[0].legend()
ax[1].plot(InclinePlane['t'], InclinePlane['y'], label='Measured', color='green')
ax[1].set_title('Y vs. Time')
ax[1].set_xlabel('Time (s)')
ax[1].set_ylabel('Position (m)')
ax[1].legend()
fig.suptitle('Position vs. Time')
plt.show()

In [ ]:
plt.figure(figsize=(12, 8))
plt.plot(freeFall['t'], freeFall['y'], label='Measured', color='green')
plt.title('Y vs. Time')
plt.xlabel('Time (s)')
plt.ylabel('Position (m)')
plt.legend()
plt.show()

In [10]:
# Non Linear Curve Fiting Functions
def NoFrictionModel(t):
        return freeFall['y'][0] + (1/2)*((g)*t**2)

def ViscousFrictionModel(t, a):
    return (g/a)*t - (((g/a)**2)/g)*(1 - np.exp(-a*t)) + freeFall['y'][0]

def TurbulentFrictionModel(t,b):
    return -(1 / b) * np.log(np.cosh(g * t / -np.sqrt(-g / b))) + freeFall['y'][0]

# Incline Plane Equations
def IncNoFrictionX(t):
    return x0 + (0.5 * -g * (t**2) * np.sin(theta) * np.cos(theta))

def IncNoFrictionY(t):
    return y0 - (0.5 * -g) * (t**2) * (np.sin(theta)**2)

def IncFrictionX(t, k):
    factor = (-g * t**2 * np.sin(theta)*np.cos(theta)) / (2 * (1 + k))
    return x0 + factor

def IncFrictionY(t, b):
    factor = (-g * t**2 * np.sin(theta)**2) / (2 * (1 + b))    
    return y0 - factor
    # return ((InclinePlane['v']**2)/g)*1.2  

def NonLinAnalysis(f_model, x, y, ParamsArr, Tweights):
    if ParamsArr is None:
        Params = None
        std = None 
        y_pred = f_model(x)
        ss_res = np.sum((y - y_pred) ** 2)
        ss_tot = np.sum((y - np.mean(y)) ** 2)
        R2 = 1 - (ss_res / ss_tot)
    else:
        Params, cov = curve_fit(f_model, x, y, p0=ParamsArr, sigma=Tweights)
        std = np.sqrt(np.diag(cov))
        y_pred = f_model(x, *Params)
        ss_res = np.sum((y - y_pred) ** 2)
        ss_tot = np.sum((y - np.mean(y)) ** 2)
        R2 = 1 - (ss_res / ss_tot)
    return Params, std, R2

def CoeffDetermin(f_model,x,y,Params):
        if Params is None:
            Params = None
            y_pred = f_model(x)
            ss_res = np.sum((y - y_pred) ** 2)
            ss_tot = np.sum((y - np.mean(y)) ** 2)
            R2 = 1 - (ss_res / ss_tot)
        else:
            y_pred = f_model(x, *Params)
            ss_res = np.sum((y - y_pred) ** 2)
            ss_tot = np.sum((y - np.mean(y)) ** 2)
            R2 = 1 - (ss_res / ss_tot)
        return R2   

In [ ]:
plt.figure(figsize=(12, 8))
NoFritParams, NoFritStd, NoFritR2 = NonLinAnalysis(NoFrictionModel,freeFall['t'],freeFall['y'],None,freeFall_Tweights)
print(f"Params: {NoFritParams}")
print(f"Std: {NoFritStd}")
print(f"R2: {NoFritR2}")
plt.plot(freeFall['t'], freeFall['y'], label='Measured', color='r')
plt.plot(freeFall['t'], NoFrictionModel(freeFall['t']), label='No Friction', color='green')
plt.title('No Friction Model')
plt.xlabel('Time (s)')
plt.ylabel('Position (m)')
NoFictData = [['r²', f"{NoFritR2:.4f}"]]
plt.table(cellText=NoFictData,
                 loc='center left',
                 cellLoc='center',
                 colWidths=[0.1, 0.1])
plt.legend()
plt.show()

# An R2 of -2.55 is right this model is just a really really bad fit.


In [ ]:
plt.figure(figsize=(12, 8))
plt.plot(freeFall['t'], freeFall['x'], label='Measured', color='green')
plt.title('X vs. Time')
plt.xlabel('Time (s)')
plt.ylabel('Position (m)')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(12, 8))
VicParams, VicStd, VicR2 = NonLinAnalysis(ViscousFrictionModel,freeFall['t'],freeFall['y'],[3.5],freeFall_Tweights)
print(f"Params: {VicParams}")
print(f"Std: {VicStd}")
print(f"R2: {VicR2}")
plt.plot(freeFall['t'], freeFall['y'], label='Measured', color='red')
plt.plot(freeFall['t'], ViscousFrictionModel(freeFall['t'], VicParams[0]), label='Viscous Friction',color='green')
VicData = [['a',f"{VicParams[0]:.4f}"],
        ['std', f"{VicStd[0]:.4f}"],
        ['r²', f"{VicR2:.4f}"]]
plt.table(cellText=VicData,
                 loc='lower left',
                 cellLoc='center',
                 colWidths=[0.1, 0.1])
plt.legend()
plt.title('Viscous Drag Model')
plt.show()

In [14]:
# plt.plot(freeFall['t'], freeFall['y'], label='No Friction', color='red')
# plt.plot(freeFall['t'], TurbulentFrictionModel(freeFall['t'], 1.7), label='Viscous Friction',color='green')
# plt.legend()
# plt.title('Y vs. Time')
# plt.show()

In [ ]:
plt.figure(figsize=(12, 8))
TurbParams, TurbStd, TurbR2 = NonLinAnalysis(TurbulentFrictionModel,freeFall['t'],freeFall['y'],[1.7],freeFall_Tweights)
print(f"Params: {TurbParams}")
print(f"Std: {TurbStd}")
print(f"R2: {TurbR2}")
plt.plot(freeFall['t'], freeFall['y'], label='Measured', color='red')
plt.plot(freeFall['t'], TurbulentFrictionModel(freeFall['t'], TurbParams[0]), label='Turbulent Friction',color='green')
plt.legend()
TurbData = [['b',f"{TurbParams[0]:.4f}"],
        ['std', f"{TurbStd[0]:.4f}"],
        ['r²', f"{TurbR2:.4f}"]]
plt.table(cellText=TurbData,
                 loc='lower left',
                 cellLoc='center',
                 colWidths=[0.1, 0.1])
plt.title('Turbulent Friction Model')
plt.show()

### Incline Experiment

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(12, 4))
IncNoFritXParams, IncNoFritXStd, IncNoFritXR2 = NonLinAnalysis(IncNoFrictionX,InclinePlane['t'],InclinePlane['x'],None,incline_Tweights)
# print(f"Params: {IncNoFritXParams}")
# print(f"Std: {IncNoFritXStd}")
print(f"X-R^2: {IncNoFritXR2}")
IncNoFritYParams, IncNoFritYStd, IncNoFritYR2 = NonLinAnalysis(IncNoFrictionY,InclinePlane['t'],InclinePlane['y'],None,None)
# print(f"\n")
# print(f"Std: {IncNoFritYStd}")
print(f"Y-R^2: {IncNoFritYR2}")
ax[0].plot(InclinePlane['t'], IncNoFrictionX(InclinePlane['t']), label='No Friction', color='red')
ax[0].plot(InclinePlane['t'], InclinePlane['x'], label='Measured', color='green')
ax[0].set_title('X vs. Time')
ax[0].set_xlabel('Time (s)')
ax[0].set_ylabel('Position (m)')
IncNoFictData0 = [['r²', f"{IncNoFritXR2:.2f}"]]
ax[0].table(cellText=IncNoFictData0,
                 loc='center left',
                 cellLoc='center',
                 colWidths=[0.1, 0.1])
ax[0].legend()
ax[1].plot(InclinePlane['t'], IncNoFrictionY(InclinePlane['t']), label='No Friction', color='red')
ax[1].plot(InclinePlane['t'], InclinePlane['y'], label='Measured', color='green')
ax[1].set_title('Y vs. Time')
ax[1].set_xlabel('Time (s)')
ax[1].set_ylabel('Position (m)')
IncNoFictData1 = [['r²', f"{IncNoFritYR2:.2f}"]]
ax[1].table(cellText=IncNoFictData1,
                 loc='center left',
                 cellLoc='center',
                 colWidths=[0.1, 0.1])
ax[1].legend()
fig.suptitle('Position vs. Time')
plt.show()

In [17]:
# plt.plot(InclinePlane['t'], IncFrictionY(InclinePlane['t'],9), label='Friction', color='red')
# plt.plot(InclinePlane['t'], InclinePlane['y'], label='Measured', color='green')
# plt.title('Y vs. Time')
# plt.xlabel('Time (s)')
# plt.ylabel('Position (m)')
# plt.legend()
# plt.show()
# plt.plot(InclinePlane['t'], IncFrictionX(InclinePlane['t'],8.89646786), label='Friction', color='red')
# plt.plot(InclinePlane['t'], InclinePlane['x'], label='Measured', color='green')
# plt.title('Y vs. Time')
# plt.xlabel('Time (s)')
# plt.ylabel('Position (m)')
# plt.legend()
# plt.show()

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(12, 4))
IncFritYParams, IncFritYStd, IncFritYR2 = NonLinAnalysis(IncFrictionY,InclinePlane['t'],InclinePlane['y'],[9],incline_Tweights)
print(f"Yk Value: {IncFritYParams}")
print(f"Y-Std: {IncFritYStd}")
print(f"Y-R^2: {IncFritYR2}\n")
IncFritXParams, IncFritXStd, IncFritXR2 = NonLinAnalysis(IncFrictionX,InclinePlane['t'],InclinePlane['x'],IncFritYParams,incline_Tweights)
print(f"Xk Value: {IncFritXParams}")
print(f"X-Std: {IncFritXStd}")
print(f"X-R^2: {IncFritXR2}\n")
k_average = np.mean([IncFritXParams[0],IncFritYParams[0]])
print(f"k-avg: {k_average}")



ax[0].plot(InclinePlane['t'], IncFrictionX(InclinePlane['t'],k_average), label='Friction', color='red')
ax[0].plot(InclinePlane['t'], InclinePlane['x'], label='Measured', color='green')
ax[0].set_title('X vs. Time')
ax[0].set_xlabel('Time (s)')
ax[0].set_ylabel('Position (m)')
IncFictData0 = [['k',f"{k_average:.4f}"],
        ['std', f"{IncFritXStd[0]:.4f}"],
        ['r²', f"{IncFritXR2:.4f}"]]
ax[0].table(cellText=IncFictData0,
                 loc='lower right',
                 cellLoc='center',
                 colWidths=[0.1, 0.1])
ax[0].legend()
IncFritYParams, IncFritYStd, IncFritYR2 = NonLinAnalysis(IncFrictionY,InclinePlane['t'],InclinePlane['y'],[k_average],incline_Tweights)
IncFritXParams, IncFritXStd, IncFritXR2 = NonLinAnalysis(IncFrictionX,InclinePlane['t'],InclinePlane['x'],[k_average],incline_Tweights)
IncFritXR2 = CoeffDetermin(IncFrictionX, InclinePlane['t'], InclinePlane['x'], [k_average])
IncFritYR2 = CoeffDetermin(IncFrictionY, InclinePlane['t'], InclinePlane['y'], [k_average])
ax[1].plot(InclinePlane['t'], IncFrictionY(InclinePlane['t'],k_average), label='Friction', color='red')
ax[1].plot(InclinePlane['t'], InclinePlane['y'], label='Measured', color='green')
ax[1].set_title('Y vs. Time')
ax[1].set_xlabel('Time (s)')
ax[1].set_ylabel('Position (m)')
IncFictData1 = [['k',f"{k_average:.4f}"],
        ['std', f"{IncFritYStd[0]:.4f}"],
        ['r²', f"{IncFritYR2:.4f}"]]
ax[1].table(cellText=IncFictData1,
                 loc='lower left',
                 cellLoc='center',
                 colWidths=[0.1, 0.1])
ax[1].legend()
fig.suptitle('Position vs. Time')
plt.show()